In [1]:
from gurobipy import *
from math import sqrt
import pandas as pd
import numpy as np

In [2]:
def phi(a, b, t):
  if(a+b>=2*t):
    result = (a-t)*(b-t)
  else:
    result = -1/2*((a-t)**2+(b-t)**2)
  return result
def phi_(a, b, t):
  if(-a+b>=2*t):
    result = (-a-b)*(b-t)
  else:
    result = -1/2*((-a-t)**2+(b-t)**2)
  return result

Regularized problem:

In [8]:
size = 200
card = 5
sigma=np.genfromtxt("orl200-005-b.mat", skip_header=1, dtype='int')
sigma=sigma[0:size,0:size]
mu=pd.read_csv("orl200-005-b.txt",sep=" ", index_col=False, skiprows=1,names=['mu_i', 'ignore'])
rho = np.genfromtxt("orl200-005-b.rho")
rho = rho.item()
mu=mu.iloc[0:size,0]
granice = pd.read_csv("orl200-005-b.bds", sep=" ", index_col=False,names=['l_i', 'u_i'])
granice=granice.transpose()
u_i=granice.iloc[1,0:size]

In [9]:
t = 1
x_guess = np.zeros(size)
y_guess = np.ones(size)
cond = 1
while ((t >= 10**(-8)) & (cond>10**(-6))):
    m = Model('portfolio')
    vars_ = pd.Series(m.addVars(size))  #C_0, ..., C_size
    y = pd.Series(m.addVars(size))
    min1 = pd.Series(m.addVars(size))
    min2 = pd.Series(m.addVars(size))
    portfolio_risk = vars_.dot(sigma).dot(vars_)
    m.setObjective(portfolio_risk, GRB.MINIMIZE)
    m.addConstr(vars_.sum() <= 1, 'budget')
    #mu=np.asarray(mu)
    m.addConstrs(((0 <= y[i]) for i in range(size)), 'yG0')
    m.addConstrs(((y[i]<=1) for i in range(size)), 'yL1')
    m.addConstrs(((0<=vars_[i]) for i in range(size)), 'u_i')
    m.addConstrs(((vars_[i] <= u_i[i]) for i in range(size)), 'u_i')
    m.addConstr(size-card <= y.sum(), 'cardinality')
    m.addConstr(rho <= mu.dot(vars_), 'portf_return')
    vars_.start=x_guess
    y.start=y_guess
    #m.addGenConstrMin(min1, [vars_[i],y[i]], "minconstr")
    m.addConstrs((min1[i] == min_(vars_[i],y[i]) for i in range(size)), 'min1')
    m.addConstrs((min1[i]<=t for i in range(size)), 'proba1')
    #m.addConstrs((min2[i] == min_(-1*vars_[i],y[i]) for i in range(size)), 'min2')
    #m.addConstrs((min2[i]<=t for i in range(size)), 'proba2')
    #m.addConstrs(((vars_[i]<=t) | (y[i]<=t) for i in range(size)), "phi")
    #m.addConstrs((min_(-vars_[i],y[i])<=t for i in range(size)), 'proba2')
    #m.addConstrs(((-vars_[i]<=t) | (y[i]<=t) for i in range(size)), "phi_")
    #m.addConstrs(((vars_[i]+y[i]>=2*t and (vars_[i]-t)*(y[i]-t)<=0)\
     #             or (vars_[i]+y[i]<2*t and -1/2*((vars_[i]-t)**2+(y[i]-t)**2)<=0)\
    #              for i in range(size)), 'phi')
    #for i in range(size):
    #    if(vars_[i]+y[i]>=2*t):
    #        m.addConstr((vars_[i]-y[i])*(y[i]-t)<=0, 'phi1')
    #    else:
    #        m.addConstr(-1/2*((vars_[i]-t)**2+(y[i]-t)**2)<=0, 'phi2')
    #for i in range(size):
    #    if(-vars_[i]+y[i]>=2*t):
    #        m.addConstr((-vars_[i]-y[i])*(y[i]-t)<=0, 'phi_1')
    #    else:
    #        m.addConstr(-1/2*((-vars_[i]-t)**2+(y[i]-t)**2)<=0, 'phi_2')
    m.update()
    m.optimize()
    x_guess= np.array([v.x for v in vars_])
    y_guess= np.array([v.x for v in y])
    t = t*0.01
    cond = max(abs(np.multiply(x_guess,y_guess)))


Optimize a model with 1003 rows, 800 columns and 1600 nonzeros
Model has 20100 quadratic objective terms
Model has 200 general constraints
Variable types: 800 continuous, 0 integer (0 binary)
Coefficient statistics:
  Matrix range     [2e-03, 1e+00]
  Objective range  [0e+00, 0e+00]
  QObjective range [4e+00, 3e+03]
  Bounds range     [0e+00, 0e+00]
  RHS range        [5e-03, 2e+02]
Presolve removed 1001 rows and 600 columns
Presolve time: 0.00s
Presolved: 2 rows, 200 columns, 400 nonzeros
Presolved model has 20100 quadratic objective terms
Variable types: 200 continuous, 0 integer (0 binary)
Presolve time: 0.06s
Presolved: 2 rows, 200 columns, 400 nonzeros
Presolved model has 20100 quadratic objective terms
Root barrier log...

Ordering time: 0.00s

Barrier statistics:
 Free vars  : 199
 AA' NZ     : 2.010e+04
 Factor NZ  : 2.030e+04
 Factor Ops : 2.727e+06 (less than 1 second per iteration)
 Threads    : 2

                  Objective                Residual
Iter       Primal        

 101347 47724   10.02216   24  187   40.29645    6.02881  85.0%  15.7  180s
 103146 48607    6.10920   24  165   40.29645    6.03060  85.0%  15.7  185s
 105355 50089   20.15103   78  133   40.29645    6.03483  85.0%  15.8  190s
 107530 50945    9.24769   25  186   40.29645    6.03684  85.0%  15.7  196s
 109530 51798   10.04828   31  180   40.29645    6.04051  85.0%  15.6  201s
 111424 52986    7.80284   46  148   40.29645    6.04238  85.0%  15.6  205s
 113077 53823   12.85024   42  169   40.29645    6.04277  85.0%  15.5  210s
 114507 54560    8.87518   58  146   40.29645    6.04277  85.0%  15.6  215s
 116039 54972   13.32307   88  123   40.29645    6.04277  85.0%  15.6  220s
 117952 56051   38.79584  140   52   40.29645    6.05141  85.0%  15.6  225s
 119950 56879   23.23556   81  127   40.29645    6.05534  85.0%  15.6  230s
 122198 57890    9.83191   25  186   40.29645    6.05917  85.0%  15.5  235s
 124476 59051   11.15213   31  180   40.29645    6.06207  85.0%  15.4  240s
 127348 5996

 311443 136595   32.28846  150   61   40.29645    6.26142  84.5%  14.3  750s
 312664 137130    8.64588   54  149   40.29645    6.26213  84.5%  14.3  755s
 315125 138153   24.10155  101  110   40.29645    6.26388  84.5%  14.3  760s
 317370 138842   35.05119  145   65   40.29645    6.26608  84.5%  14.2  765s
 319418 139389    6.92128   36  156   40.29645    6.26685  84.4%  14.3  770s
 321465 140251    8.76847   57  145   40.29645    6.26685  84.4%  14.3  775s
 323286 140811   12.25843   82  129   40.29645    6.26685  84.4%  14.3  780s
 325228 141479   38.34968  142   55   40.29645    6.26685  84.4%  14.3  785s
 327098 142365    9.78193   63  144   40.29645    6.27379  84.4%  14.3  790s
 327590 142602    8.54819   53  150   40.29645    6.27412  84.4%  14.3  799s
 327870 142842   22.04946  118   93   40.29645    6.27420  84.4%  14.3  800s
 329940 143653    7.27553   42  152   40.29645    6.27479  84.4%  14.3  805s
 331930 144425    9.91333   67  140   40.29645    6.27479  84.4%  14.3  810s

 503724 209532     cutoff   30        40.29645    6.39597  84.1%  14.3 1311s
 505525 210051   10.19479   31  180   40.29645    6.39747  84.1%  14.3 1315s
 507023 210647   10.97951   31  180   40.29645    6.39828  84.1%  14.3 1320s
 508879 211212   35.77062  124   66   40.29645    6.39918  84.1%  14.3 1325s
 511059 212303   11.62190   30  181   40.29645    6.40022  84.1%  14.3 1330s
 512525 212733   12.68160   35  176   40.29645    6.40138  84.1%  14.3 1335s
 514413 213521    8.18622   52  149   40.29645    6.40312  84.1%  14.4 1340s
 516709 214495   30.45598  145   66   40.29645    6.40491  84.1%  14.4 1345s
 518736 215206   23.39339   79  124   40.29645    6.40550  84.1%  14.4 1350s
 520418 215836    6.45095   29  161   40.29645    6.40606  84.1%  14.3 1355s
 522708 216629     cutoff   64        40.29645    6.40803  84.1%  14.3 1360s
 524453 217178   11.32528   28  183   40.29645    6.40884  84.1%  14.3 1374s
 524747 217241    6.53352   31  159   40.29645    6.40884  84.1%  14.3 1375s

AttributeError: b"Unable to retrieve attribute 'x'"

In [5]:
for v in vars_:
    if v.x > 0:
        print('\t%s\t: %g' % (v.varname, v.x))

	C0	: 0.0964341
	C4	: 0.102496
	C5	: 0.11615
	C6	: 0.153309
	C8	: 0.11514
